In [14]:
import requests

import torch
from PIL import Image
from transformers import AutoProcessor, AutoModelForZeroShotObjectDetection, AutoModelForCausalLM


In [ ]:
#model used for captioning image input query
input_processor = AutoProcessor.from_pretrained("microsoft/git-base-coco")
input_model = AutoModelForCausalLM.from_pretrained("microsoft/git-base-coco")
#model used for object detection
dino_processor = AutoProcessor.from_pretrained("IDEA-Research/grounding-dino-tiny")
dino_model = AutoModelForZeroShotObjectDetection.from_pretrained("IDEA-Research/grounding-dino-tiny").to("cuda")

In [16]:
input_url = "input-data/cat.jpg"
image_url = "http://images.cocodataset.org/val2017/000000039769.jpg"

In [17]:
#opens the image input, and captions it
image = Image.open(input_url)

pixel_values = input_processor(images=image, return_tensors="pt").pixel_values

generated_ids = input_model.generate(pixel_values=pixel_values, max_length=50)
generated_caption = input_processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_caption)


image = Image.open(requests.get(image_url, stream=True).raw)
# Uses generated caption to detect
text_labels = [[generated_caption, "null"]]

inputs = dino_processor(images=image, text=text_labels, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = dino_model(**inputs)

results = dino_processor.post_process_grounded_object_detection(
    outputs,
    inputs.input_ids,
    box_threshold=0.4,
    text_threshold=0.3,
    target_sizes=[image.size[::-1]]
)

# Retrieve the first image result
result = results[0]
for box, score, labels in zip(result["boxes"], result["scores"], result["labels"]):
    box = [round(x, 2) for x in box.tolist()]
    print(f"Detected {labels} with confidence {round(score.item(), 3)} at location {box}")

red cat sitting on a white background
Detected red cat with confidence 0.554 at location [345.69, 23.8, 637.67, 372.95]
